# Lesson 2: RAG Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Import OpenAI API key from prompt user input on keypass

import os
# import sys
# Get API key from user input

api_key = input("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = api_key

# Get HuggingFace API key from user input
huggingface_key = input("Enter your HuggingFace API key: ")
os.environ["HUGGINGFACE_API_KEY"] = huggingface_key


In [4]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [5]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [6]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [8]:
from llama_index import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [9]:
from utils import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [10]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [11]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'To create your AI portfolio, you should focus on building a collection of projects that demonstrate your skill progression. This can be achieved by starting with simpler projects and gradually increasing the complexity as you gain more experience. Additionally, it is important to ensure that your portfolio showcases a variety of AI techniques and technologies to highlight your versatility. By doing so, you can effectively demonstrate your capabilities to potential employers and increase your chances of securing an AI job.'

## Feedback functions

In [12]:
import nest_asyncio

nest_asyncio.apply()

In [13]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [14]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [15]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [16]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [17]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### 3. Groundedness

In [18]:
from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=provider)

In [19]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [20]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [21]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [22]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [23]:
eval_questions.append("How can I be successful in AI?")

In [24]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [25]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

boto3,botocore is/are required for using BedrockEndpoint. You should be able to install it/them with
	pip install boto3 botocore


In [26]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b5621c2350e4c748134bd3033e221038,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_b5621c2350e4c748134...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:41:33.337411"", ""...",2023-12-12T00:41:35.309863,1.0,1.00,0.40,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'source': 'Chapter 7: A Simple Fram...,[{'args': {'question': 'What are the keys to b...,1,514,0.000788
1,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_632b897da6eaea3a74f10bb2c8a22b76,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_632b897da6eaea3a74f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:41:35.400161"", ""...",2023-12-12T00:41:39.090218,1.0,0.95,0.75,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'To get a project starte...,[{'args': {'question': 'How can teamwork contr...,3,662,0.001045
2,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5fc46cfbb1d0a7abccba0fb26d0bc72f,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_5fc46cfbb1d0a7abccb...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:41:39.179534"", ""...",2023-12-12T00:41:42.510290,1.0,0.40,0.10,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'What is the hiring proce...,[{'args': {'question': 'What is the importance...,3,530,0.000843
3,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b67b9f564ef98023095596ffb083ff98,"""What are some good habits to develop for a su...","""Developing good habits in eating, exercise, s...",-,"{""record_id"": ""record_hash_b67b9f564ef98023095...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:41:42.598892"", ""...",2023-12-12T00:41:45.397693,1.0,0.50,0.30,[{'args': {'prompt': 'What are some good habit...,"[{'args': {'source': '(For example, many arti...",[{'args': {'question': 'What are some good hab...,2,447,0.000696
4,App_1,"{""app_id"": ""App_1"", ""tags"": ""-"", ""metadata"": {...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_2435fb9b53a920ff1d16c813a551ae06,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_2435fb9b53a920ff1d1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-12T00:41:45.484576"", ""...",2023-12-12T00:41:49.206342,1.0,NaN,NaN,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,3,571,0.000912


In [27]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Answer Relevance,Groundedness
0,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.""",0.40,1.0,1.00
1,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.""",0.75,1.0,0.95
2,"""What is the importance of networking in AI?""","""Networking is important in AI because it allows individuals to connect with others who have experience and knowledge in the field. By reaching out to people in their network, such as friends or alumni, individuals can receive guidance and support as they navigate their career in AI. Additionally, attending meetups and engaging in informational interviews can help individuals expand their network and gain insights from professionals in the industry. This networking can provide valuable opportunities for learning, mentorship, and potential job prospects in the future.""",0.10,1.0,0.40
3,"""What are some good habits to develop for a successful career?""","""Developing good habits in eating, exercise, sleep, personal relationships, work, learning, and self-care can be beneficial for a successful career. These habits can help individuals move forward while staying healthy and contribute to their overall success in their professional lives.""",0.30,1.0,0.50
4,"""How can altruism be beneficial in building a career?""","""Altruism can be beneficial in building a career by helping others while also advancing one's own career. By aiming to lift others during every step of their own journey, individuals can achieve better outcomes for themselves. This can be done by offering support, guidance, and assistance to others in their professional endeavors. By helping others succeed, individuals can build strong relationships, expand their network, and gain a positive reputation in their field. Additionally, practicing altruism can also contribute to personal growth and fulfillment, which can ultimately enhance one's career satisfaction and success.""",NaN,1.0,NaN
5,"""What is imposter syndrome and how does it relate to AI?""","""Imposter syndrome is a psychological phenomenon where individuals doubt their own abilities and fear being exposed as a fraud, despite evidence of their success. In the context of AI, newcomers to the field sometimes experience imposter syndrome, questioning whether they truly belong in the AI community and if they are capable of contributing. It is important to address imposter syndrome in order to encourage individuals to continue growing in AI and not let self-doubt hinder their progress.""",NaN,0.9,NaN
6,"""Who are some accomplished individuals who have experienced imposter syndrome?""","""Sheryl Sandberg, Michelle Obama, Tom Hanks, and Mike Cannon-Brookes are some accomplished individuals who have experienced imposter syndrome.""",NaN,1.0,NaN
7,"""What is the first step to becoming good at AI?""","""The first step to becoming good at AI is to learn foundational technical skills.""",NaN,1.0,NaN
8,"""What are some common challenges in AI?""","""Some common challenges in AI include keeping up-to-date with evolving technology, finding suitable projects and estimating timelines and return on investment, managing the highly iterative nature of AI projects, collaborating wi

In [28]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
App_1,0.407143,0.981818,0.77,2.272727,0.000876


In [29]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.21:8503 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>